ChainerMNのプログラムについて、[MNIST](https://en.wikipedia.org/wiki/MNIST_database)(手書き数字認識)のサンプルを用いて説明します。<br>
ここでは、以下のモジュールがロードされていることを前提とします。

In [ ]:
from __future__ import print_function

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

import chainermn

まず、ネットワークのモデルを定義します。<br>
ここでは、各層が1000個のユニットで出力が10個のである単純な3層のネットワーク(MLP: Multi Layer Perceptron)を用います。<br>
\__init\__でネットワークの接続を定義し、\__call\__でデータの流れを定義しています。<br>
ネットワークのモデルの定義に関してはChainerと同様で、詳細は以下のサイトを参照してください。<br>
http://docs.chainer.org/en/latest/tutorial/index.html

In [ ]:
class MLP(chainer.Chain):

    def __init__(self, n_units, n_out):
        super(MLP, self).__init__(
            l1=L.Linear(784, n_units),  
            l2=L.Linear(n_units, n_units),
            l3=L.Linear(n_units, n_out),
        )

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)
    
model = L.Classifier(MLP(1000, 10))

ChainerMNはデータ並列型の分散処理を採用しています。各ワーカプロセスがモデルを保持し、バッチ（データセット）に対して勾配を計算し、計算結果を用いてワーカプロセス間で学習モデルの更新を行うことで分散深層学習を行います。具体的には、Chainerで行っていたForward, Backward, Optimizeの処理を分散化し、BackwardとOptimizeの間にAllReduce(各ワーカプロセスの結果を集約して集約した結果を各ワーカプロセスへ送付する)を行うことで実現しています。この分散化にはMPI(Message Passing Interface)と呼ばれる高性能計算(HPC)で一般的に使われている実装が用いられています。詳細については以下のサイトを参照してください。<br>
http://chainermn.readthedocs.io/en/latest/tutorial/overview.html


ChainerMNで分散深層学習を行うためには、まず、Communicatorを作成します。<br>
Communicatorはワーカプロセス間の通信を担います。ワーカプロセスには0番からn(並列数)番までのRankと呼ばれる番号が割り当てられます。<br>
GPUを用いて実行する場合は、create_communicatorを用いて以下のようにCommunicatorを作成します。

In [ ]:
comm = chainermn.create_communicator('hierarchical')
device = comm.intra_rank

また、GPUの使用の宣言をし、modelをGPUへ送る必要があります。

In [ ]:
chainer.cuda.get_device(device).use()
model.to_gpu()

CPUのみを用いて実行する場合は以下のようにCommunicatorを作成します。

In [ ]:
comm = chainermn.create_communicator('naive')
device = -1

次に、Multi-Node Optimizerを作成します。<br>
create_multi_node_optimizerにChainerのOptimizerとCommunicatorを与えることで、分散対応したOptimizerを作成します。<br>
ChainerMNのOptimizerはChainerのOptimizerと同様に扱うことができます。

In [ ]:
optimizer = chainermn.create_multi_node_optimizer(chainer.optimizers.Adam(), comm)

ネットワークのモデルをOptimizerに設定します。

In [ ]:
optimizer.setup(model)

さらに、学習データの準備をします。<br>
ここでは、MNISTのデータを取得し、各ワーカプロセスに配布します。<br>
まず、MNISTの訓練データ、テストデータを取得します。<br>
ワーカプロセスが0番のときはMNISTのデータを取得し、その他の場合は何もしません。

In [ ]:
if comm.rank == 0:
    train, test = chainer.datasets.get_mnist()
else:
    train, test = None, None

その後、0番のワーカプロセスが取得したMNISTのデータを他のワーカプロセスに配布します。

In [ ]:
train = chainermn.scatter_dataset(train, comm)
test = chainermn.scatter_dataset(test, comm)

次に、各ワーカプロセスで訓練データとテストデータのIteratorを作成します。今、ミニバッチサイズを100とします。

In [ ]:
batchsize = 100
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

学習処理を実際に行うTrainerの設定を行います。ここでは、20 epochを実行し、resultに出力します。<br>
Updaterに各種最適化手法を設定することができます。詳細は以下のサイトを参照してください。<br>
http://docs.chainer.org/en/latest/tutorial/index.html

In [ ]:
updater = training.StandardUpdater(train_iter, optimizer, device=device)
trainer = training.Trainer(updater, (epoch, 'epoch'), out='result')

訓練データを基に学習したモデルを分散して評価するために、MultinodeEvaluationrを作成します。

In [ ]:
evaluator = extensions.Evaluator(test_iter, model, device=device)
evaluator = chainermn.create_multi_node_evaluator(evaluator, comm)
trainer.extend(evaluator)
if comm.rank == 0:
    trainer.extend(extensions.dump_graph('main/loss'))
    trainer.extend(extensions.LogReport())
    trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss', 
         'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))
    trainer.extend(extensions.ProgressBar())

ここまでで準備が揃ったので、実行します。

In [ ]:
trainer.run()